## This notebook is dedicated to the exploration of the Lyrics dataset


In [ ]:
import pandas as pd


In [ ]:
n_rows = 40000
lyrics = pd.read_csv('../song_lyrics.csv', nrows=n_rows)
# lyrics[:10, 1:]

In [ ]:
#lyrics.iloc[:10, 2:8]
print(lyrics.iat[9,6])

In [ ]:
lyrics.iloc[:10 ]

## Remove brackets
Here we remove the brackets from the lyrics, that they can be used by the classifiers


In [ ]:
import re
def preprocess(song):
    output_text = re.sub(r'\[\s*.*?\s*\]\n', '', song)
    return output_text

In [ ]:
# Retrieve the content of the cell
import string
cell_content = lyrics.iat[5000,6]  # Replace with the appropriate row and column labels
# print(str(cell_content))
# Specify the file path and name for the text file
file_path = '../files/lyrics.txt'

# Write the cell content to the text file
with open(file_path, 'w') as file:
    lines = str(cell_content).splitlines()
    for line in lines:
        if line.strip():  # Check if the line is non-blank
            # print(line)
            last_two_words = re.findall(r'\b(\w+\W*\w+)\W*$', line)[-1]
            last_two_words = re.sub(r'\d', '', last_two_words)  # Remove numbers
            last_two_words = last_two_words.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
            file.write(last_two_words + '\n')

Here we remove the lyrics of the other languages that are not english, we drop each row which contains such
and we reindex the table afterwards

In [ ]:

for i, row in lyrics.iterrows():
    if row['language'] != 'en':
        lyrics.drop(i, inplace=True)
        # print('Hey it is not an english lyrics!')
    else:
        cleaned = preprocess(row['lyrics'])
        lyrics.at[i, 'lyrics'] = cleaned


lyrics.reset_index(drop=True, inplace=True)
# lyrics.iloc[:10]

In [ ]:
print(lyrics.shape)

## Rhyme extraction
What could be the desired marked formats?
1. Most frequent scheme - ABAB, ABBA (underfitting)
2. The exact words that are being rhymed?
3. The whole rhyming scheme of the song - ABABCDDC-FFFF-ABABCDCD-FFFF-FFFF (overfitting)
4. Multisyllable rhymes - (AB)(AB)(CB)(CB)

In [27]:
# Specify the file path and name
file_path = '../cmudict-0.7b.txt'

# Create an empty dictionary to store the word-phoneme mappings
word_phoneme_dict = {}

encodings = ['utf-8', 'latin-1', 'utf-16', 'cp1252']
for encoding in encodings:
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line:
                    split = line.split()
                    word_phoneme_dict[split[0]] = ' '.join(split[1:])
            # break
    except UnicodeDecodeError:
        print('Error')

# Print the dictionary
#print(word_phoneme_dict)

Error
Error
Error
Error


In [28]:
print(word_phoneme_dict.get('FOOTBALL'))
print(len(word_phoneme_dict))
# print(word_phoneme_dict)

None
35129


In [29]:
import psutil

# Get the current available memory in bytes
available_memory = psutil.virtual_memory().available

# Convert bytes to human-readable format
available_memory_gb = available_memory / (1024 ** 3)  # Convert bytes to gigabytes

# Print the available memory
print(f"Available Memory: {available_memory_gb:.2f} GB")


Available Memory: 5.97 GB


In [24]:
# here a database will be created to store the key-value pairs
!pip install SQLAlchemy

You should consider upgrading via the '/Users/zhenyabudnyk/PycharmProjects/LyrAIX/venv/bin/python -m pip install --upgrade pip' command.


In [55]:
from sqlalchemy import create_engine, Column, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Create the SQLAlchemy engine
engine = create_engine('sqlite:///word_phonemes.db')
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

In [56]:
from sqlalchemy import Column, Integer, String

class KeyValue(Base):
    __tablename__ = 'word_phonemes'

    id = Column(Integer, primary_key=True, autoincrement=True)
    key = Column(String)
    value = Column(String)

    def __init__(self, key, value):
        self.key = key
        self.value = value

Base.metadata.create_all(engine)

In [57]:
file_path = '../cmudict-0.7b.txt'

# Create an empty dictionary to store the word-phoneme mappings
word_phoneme_dict = {}

encodings = ['utf-8', 'latin-1', 'utf-16', 'cp1252']
for encoding in encodings:
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line:
                    split = line.split()
                    # word_phoneme_dict[split[0]] = ' '.join(split[1:])
                    key_value = KeyValue(key=split[0], value=' '.join(split[1:]))
                    session.add(key_value)
            # break
    except UnicodeDecodeError:
        print('Error')


Error
Error
Error
Error


In [58]:
session.commit()
session.close()
engine.dispose()

## Topic Modelling and Extraction


## Affect labeling